In [1]:
import datetime
import numpy as np
import pandas as pd


In [ ]:
#! pip install scikit-learn==1.3.0

## 2. Choisir le bon estimateur/algorithme pour votre problème

Une fois que vos données sont prêtes, l'étape suivante consiste à choisir un algorithme ou un modèle d'apprentissage automatique approprié pour trouver des modèles dans vos données.

Quelques points à noter :
* Scikit-Learn fait référence aux modèles et algorithmes d'apprentissage automatique en tant qu'estimateurs.
* **Problème de classification** - prédire une catégorie (maladie cardiaque ou non).
     * Parfois, vous verrez `clf` (abréviation de classificateur) utilisé comme nom de variable d'une instance d'estimateur de classification.
* **Problème de régression** - prédire un nombre (prix de vente d'une voiture).
* **Problème non supervisé (données sans étiquettes)** - clustering (regroupement d'échantillons non étiquetés avec d'autres échantillons similaires non étiquetés).

Si vous savez sur quel type de problème vous travaillez, l'un des prochains endroits que vous devriez examiner est le [Scikit-Learn algorithm cheatsheet](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).

Cette aide-mémoire vous donne un aperçu de l'algorithme que vous souhaiterez peut-être utiliser pour le problème sur lequel vous travaillez.

Il est important de se rappeler que vous n’avez pas besoin de savoir explicitement ce que fait chaque algorithme à l’intérieur pour commencer à les utiliser.

Si vous commencez à appliquer différents algorithmes mais qu'ils ne semblent pas fonctionner (pas aussi bien que vous le souhaiteriez), c'est à ce moment-là que vous commencerez à approfondir chacun d'entre eux.

Jetons un coup d'œil à l'aide-mémoire et suivons-le pour découvrir certains des problèmes sur lesquels nous travaillons.

<img src="../docs/images/sklearn-ml-map.png" width=700/>

Vous pouvez voir qu’il est divisé en quatre catégories principales. Régression, classification, clustering et réduction de dimensionnalité. Chacun a son propre objectif, mais l'équipe Scikit-Learn a conçu la bibliothèque de manière à ce que les flux de travail pour chacun soient relativement similaires.

### 2.1 Choisir un modèle d'apprentissage automatique pour un problème de régression

Commençons par un problème de régression (essayer de prédire un nombre). Nous utiliserons le [ensemble de données California Housing](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) intégré au module « datasets » de Scikit-Learn.

L'objectif de l'ensemble de données California Housing est de prédire la valeur médiane d'une maison d'un quartier donné (en centaines de milliers de dollars) sur des éléments tels que l'âge de la maison, le nombre de pièces, le nombre de chambres, le nombre de personnes vivant dans la maison et plus.

In [4]:
# Obtenez l'ensemble de données sur le logement en Californie
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing; # est téléchargé sous forme de dictionnaire

Puisqu'il se trouve dans un dictionnaire, transformons-le en DataFrame afin de mieux l'inspecter.

In [5]:
housing_df = pd.DataFrame(housing["data"], columns=housing["feature_names"])
housing_df["target"] = pd.Series(housing["target"])
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [6]:
# Combien d'exemples ?
len(housing_df)

20640

Magnifique, notre objectif ici est d'utiliser les colonnes de fonctionnalités, telles que :
* `MedInc` - revenu médian dans le groupe de blocs
* `HouseAge` - âge médian de la maison dans le groupe de blocs
* `AveRooms` - nombre moyen de pièces par ménage
* `AveBedrms` - nombre moyen de chambres par ménage

Prédire la colonne « cible » qui exprime la valeur médiane de la maison pour des districts spécifiques de Californie en centaines de milliers de dollars (par exemple 4,526 = 452 600 $).

Essentiellement, chaque ligne correspond à un district différent de Californie (les données) et nous essayons de construire un modèle pour prédire la valeur médiane des maisons dans cette zone (la cible/l'étiquette) en fonction d'une série d'attributs sur les maisons de ce district. .

Puisque nous disposons de données et d’étiquettes, il s’agit d’un problème d’apprentissage supervisé.

Et puisque nous essayons de prédire un nombre, c'est un problème de régression.

Sachant ces deux choses, comment s'alignent-elles sur l'aide-mémoire de l'algorithme d'apprentissage automatique Scikit-Learn ?

<img src="../docs/images/sklearn-ml-map-cheatsheet-california-housing-ridge.png" width=700/>

En suivant la carte, sachant ce que nous savons, cela nous suggère d'essayer [`RidgeRegression`](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression). Vérifions-le.

In [7]:
# Importez la classe de modèle Ridge depuis le module Linear_model
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

# Configurer une graine aléatoire
np.random.seed(42)

# Divisez les données en fonctionnalités (X) et étiquettes (y)
X = housing_df.drop("target", axis=1)
y = housing_df["target"]

# Divisé en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instaurer et ajuster le modèle (sur l'ensemble d'entraînement)
model = Ridge()
model.fit(X_train, y_train)

# Vérifier le score du modèle (sur l'ensemble de test)
# La métrique score() par défaut des algorithmes de régression est R^2
model.score(X_test, y_test)

0.5758549611440126

Et si « RidgeRegression » ne fonctionnait pas ? Et si nous voulions améliorer nos résultats ?

<img src="../docs/images/sklearn-ml-map-cheatsheet-california-housing-ensemble.png" width=700/>

En suivant le diagramme, l'étape suivante serait d'essayer [`EnsembleRegressors`](https://scikit-learn.org/stable/modules/ensemble.html).

Ensemble est un autre mot désignant plusieurs modèles réunis pour prendre une décision.

L'une des méthodes d'ensemble les plus courantes et les plus utiles est la [Random Forest](https://scikit-learn.org/stable/modules/ensemble.html#forest). Connu pour ses temps de formation et de prédiction rapides et son adaptabilité à différents problèmes.

Le principe de base de Random Forest est de combiner un certain nombre d’arbres de décision différents, chacun aléatoirement les uns par rapport aux autres, et de faire une prédiction sur un échantillon en faisant la moyenne du résultat de chaque arbre de décision.

Une discussion approfondie de l'algorithme Random Forest dépasse le cadre de ce cahier, mais si vous souhaitez en savoir plus, [Une implémentation et une explication de la forêt aléatoire en Python](https://towardsdatascience.com/an- implémentation-et-explication-de-la-forêt-aléatoire-dans-python-77bf308a9b76) de Will Koehrsen est une excellente lecture.

Puisque nous travaillons avec la régression, nous utiliserons [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) de Scikit-Learn.

Nous pouvons utiliser exactement le même flux de travail que ci-dessus. Sauf pour changer de modèle.

In [8]:
# Importez la classe modèle RandomForestRegressor depuis le module ensemble
from sklearn.ensemble import RandomForestRegressor

# Configurer une graine aléatoire
np.random.seed(42)

# Divisez les données en fonctionnalités (X) et étiquettes (y)
X = housing_df.drop("target", axis=1)
y = housing_df["target"]

# Divisé en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instaurer et ajuster le modèle (sur l'ensemble d'entraînement)
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Vérifier le score du modèle (sur l'ensemble de test)
# La métrique de score par défaut des algorithmes de régression est R^2
model.score(X_test, y_test)

0.8066196804802649

Nous obtenons une bonne augmentation du score sur l’ensemble de tests en changeant de modèle.

Il s'agit d'un autre concept extrêmement important dans l'apprentissage automatique : si au début quelque chose ne donne pas ce que vous souhaitez, *expérimentez, expérimentez, expérimentez !*

Au début, le diagramme de l'algorithme Scikit-Learn peut sembler déroutant.

Mais une fois que vous aurez un peu de pratique en appliquant différents modèles à différents problèmes, vous commencerez à déterminer quels types d'algorithmes fonctionnent le mieux avec différents types de données.

### 2.2 Choisir un modèle d'apprentissage automatique pour un problème de classification

Voyons maintenant le processus de choix pour un problème de classification.

Supposons que vous essayiez de prédire si un patient souffrait ou non d'une maladie cardiaque en fonction de son dossier médical.

L'ensemble de données dans `../data/heart-disease.csv` contient des données uniquement pour ce problème.

In [9]:
heart_disease = pd.read_csv("../data/heart-disease.csv")
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [10]:
# Combien il-y-a t'il d'exemples ?
len(heart_disease)

303

Semblable à l'ensemble de données California Housing, nous souhaitons ici utiliser toutes les données disponibles pour prédire la colonne cible (1 pour si un patient souffre d'une maladie cardiaque et 0 pour le cas contraire).

Alors, que savons-nous?

Nous avons 303 échantillons (1 ligne = 1 échantillon) et nous essayons de prédire si un patient souffre ou non d'une maladie cardiaque.

Parce que nous essayons de prédire si chaque échantillon est une chose ou une autre, nous avons un problème de classification.

Voyons comment cela s'aligne avec notre [Scikit-Learn algorithm cheat-sheet](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).

<img src="../docs/images/sklearn-ml-map-cheatsheet-heart-disease-linear-svc.png" width=700/>

Suite à l'aide-mémoire, nous nous retrouvons à [`LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) qui signifie Classificateur de Vecteur de Support Linéaire. Essayons-le sur nos données.

In [11]:
# Importez LinearSVC du module svm
from sklearn.svm import LinearSVC

# Configurer une graine aléatoire
np.random.seed(42)

# Divisez les données en X (caractéristiques/données) et y (cible/étiquettes)
X = heart_disease.drop("target", axis=1)
y = heart_disease["target"]

# Divisé en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instancier et ajuster le modèle (sur l'ensemble d'entraînement)
clf = LinearSVC(max_iter=1000, # itérations sur les données, 1 000 est la valeur par défaut
                dual=False)
clf.fit(X_train, y_train)

# Vérifier le score du modèle (sur l'ensemble de test)
clf.score(X_test, y_test)

0.8688524590163934

Dès la sortie de la boîte (sans réglage ni amélioration), notre modèle atteint une précision de plus de 85 % !

Bien qu'il s'agisse d'un résultat sensationnel au départ, regardons le diagramme et voyons quels autres modèles nous pourrions utiliser.

<img src="../docs/images/sklearn-ml-map-cheatsheet-heart-disease-ensemble.png" width=700/>

En suivant le chemin (et en en sautant quelques-uns, ne vous inquiétez pas, nous y reviendrons), nous arrivons à [`EnsembleMethods`](https://scikit-learn.org/stable/modules/ensemble.html) encore. 

Sauf que cette fois, nous examinerons des classificateurs d'ensemble plutôt que des régresseurs.

Rappelez-vous notre [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) d'en haut?

Nous aurons un partenaire de danse, [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) qui est un modèle d'apprentissage automatique basé sur un ensemble pour la classification.

Vous pourrez peut-être deviner à quoi nous pouvons l'utiliser (indice : problèmes de classification).

Essayons!

In [12]:
# Importez la classe modèle RandomForestClassifier depuis le module ensemble
from sklearn.ensemble import RandomForestClassifier

# Configurer une graine aléatoire
np.random.seed(42)

# Divisez les données en X (caractéristiques/données) et y (cible/étiquettes)
X = heart_disease.drop("target", axis=1)
y = heart_disease["target"]

# Divisé en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instancier et ajuster le modèle (sur l'ensemble d'entraînement)
clf = RandomForestClassifier(n_estimators=100) # 100 est la valeur par défaut, mais vous pouvez essayer 1000 et voir ce qui se passe
clf.fit(X_train, y_train)

# Vérifiez le score du modèle (sur l'ensemble de test)
clf.score(X_test, y_test)

0.8524590163934426

Il semble que les hyperparamètres par défaut de « RandomForestClassifier » ne fonctionnent pas aussi bien que « LinearSVC ».

En plus d'essayer un autre modèle de classification, nous pourrions commencer à mener des expériences pour essayer d'améliorer ces modèles via un [réglage des hyperparamètres](http://scikit-learn.org/stable/modules/grid_search.html).

Le réglage des hyperparamètres est un terme sophistiqué pour ajuster certains paramètres d'un modèle afin d'essayer de l'améliorer.

Cela se produit généralement une fois que vous avez trouvé un modèle de base décent que vous souhaitez améliorer.

Dans ce cas, nous pourrions prendre soit le `RandomForestClassifier`, soit le `LinearSVC` et essayer de l'améliorer avec le réglage des hyperparamètres (que nous verrons plus tard).

Par exemple, vous pouvez essayer de prendre le paramètre « n_estimators » (le nombre d'arbres dans la forêt) de « RandomForestClassifier » et le changer de « 100 » (par défaut) à « 1000 » et voir ce qui se passe.

### Et les autres modèles ?

En regardant l'aide-mémoire de l'aglorithme Scikit-Learn et les exemples ci-dessus, vous avez peut-être remarqué que nous avons ignoré quelques modèles.

Pourquoi?

La première raison est le temps.

Couvrir chacun d’entre eux prendrait un peu plus de temps que ce que nous avons fait ici. Et le deuxième est l’efficacité des méthodes d’ensemble.

Une petite information pour la modélisation en apprentissage automatique :
* Si vous disposez de données structurées (tableaux, feuilles de calcul ou dataframes), utilisez des méthodes d'ensemble, telles qu'une forêt aléatoire.
* Si vous disposez de données non structurées (texte, images, audio, éléments ne figurant pas dans des tableaux), utilisez le deep learning ou l'apprentissage par transfert (voir le ZTM [TensorFlow](https://dbourke.link/ZTMTFcourse?ref=mrdbourke.com) et [PyTorch](https://dbourke.link/ZTMpyTorch?ref=mrdbourke.com) cours pour en savoir plus sur l'apprentissage profond).

Pour ce notebook, nous nous concentrons sur les données structurées, c'est pourquoi Random Forest a été notre modèle de choix.

Si vous souhaitez en savoir plus sur Random Forest et pourquoi c'est le cheval de guerre de l'apprentissage automatique, consultez ces ressources :
* [Wikipédia de la forêt aléatoire](https://en.wikipedia.org/wiki/Random_forest)
* [Une implémentation et une explication de la forêt aléatoire en Python](https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76) par Will Koehrsen

### Expérimentez jusqu'à ce que quelque chose fonctionne

Ce qui est beau, c'est que la façon dont l'API Scikit-Learn est conçue, une fois que vous connaissez le fonctionnement d'un modèle, en utiliser un autre est à peu près la même.

Et comme une grande partie du métier d'ingénieur en apprentissage automatique ou de data scientist consiste à expérimenter, vous voudrez peut-être essayer certains des autres modèles de l'aide-mémoire et voir comment vous vous en sortez. Plus vous réduisez le temps entre les expériences, mieux c’est.